# ULMFIT

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from exp.nb_12a import *

## Data

We load the data from 12a, instructions to create that file are there if you don't have it yet so go ahead and see.

In [3]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [4]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))

In [5]:
bs,bptt = 128,70
data = lm_databunchify(ll, bs, bptt)

In [6]:
vocab = ll.train.proc_x[1].vocab

## Finetuning the LM

Before tackling the classification task, we have to finetune our language model to the IMDB corpus.  
  
We have pretrained a samll model on [wikitext 103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) that you can download by uncommenting the following cell.

In [7]:
! wget http://files.fast.ai/models/wt103_tiny.tgz -P {path}
! tar xf {path}/wt103_tiny.tgz -C {path}

--2020-07-23 19:17:36--  http://files.fast.ai/models/wt103_tiny.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75482451 (72M) [application/x-gtar-compressed]
Saving to: ‘/home/jupyter/.fastai/data/imdb/wt103_tiny.tgz.12’

wt103_tiny.tgz.12   100%[===================>]  71.99M  14.0MB/s    in 13s     

2020-07-23 19:17:49 (5.65 MB/s) - ‘/home/jupyter/.fastai/data/imdb/wt103_tiny.tgz.12’ saved [75482451/75482451]



In [8]:
dps =  tensor([0.1, 0.15, 0.25, 0.02, 0.2]) * 0.5
tok_pad = vocab.index(PAD)

In [9]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, *dps)

In [10]:
old_wgts = torch.load(path/'pretrained'/'pretrained.pth')
old_vocab = pickle.load(open(path/'pretrained'/'vocab.pkl', 'rb'))

In our current vocabulary, it is very unlikely that the ids correspond to what is in the vocabulary used to train the pretrained model. The tokens are sorted by frequency(apart from the special tokens that are all first) so that order is specific to the corpus used. For instance, the word 'house' has different ids in the our current vocab and the pretrained one.

In [11]:
idx_house_new, idx_house_old = vocab.index('house'), old_vocab.index('house')
idx_house_new, idx_house_old 

(349, 230)

We somehow need to match our pretrained weights to the new vocabulary. This is done on the embeddings and the decoder(since the weights between embeddings and decoders are tied) by putting the rows of the embedded matrix(or decoder bias) in the right order.
  
It may also happen that we have words that aren't in the pretrained vocab, in this case, we put the mean of the pretrained embedding weights/decoder bias.

In [12]:
# old_wgts

In [13]:
len(old_wgts)

14

In [14]:
house_wgt = old_wgts['0.emb.weight'][idx_house_old]
house_bias = old_wgts['1.decoder.bias'][idx_house_old]
house_wgt.shape, house_bias.shape

(torch.Size([300]), torch.Size([]))

In [15]:
def match_embeds(old_wgts, old_vocab, new_vocab):
    wgts = old_wgts['0.emb.weight']
    bias = old_wgts['1.decoder.bias']
    wgts_m, bias_m = wgts.mean(dim=0), bias.mean()
    new_wgts = wgts.new_zeros(len(new_vocab), wgts.size(1))
    new_bias = bias.new_zeros(len(new_vocab))
    otoi = {v:k for k,v in enumerate(old_vocab)}
    for i,w in enumerate(new_vocab):
        if w in otoi:
            idx = otoi[w]
            new_wgts[i], new_bias[i] = wgts[idx], bias[idx]
        else: new_wgts[i], new_bias[i] = wgts_m, bias_m
    old_wgts['0.emb.weight']   = new_wgts
    old_wgts['0.emb_dp.emb.weight'] = new_wgts
    old_wgts['1.decoder.weight'] = new_wgts
    old_wgts['1.decoder.bias'] = new_bias
    return old_wgts

In [16]:
wgts = match_embeds(old_wgts, old_vocab, vocab)

In [17]:
len(wgts)

14

Now let's check that the word 'house' was properly converted.

In [18]:
test_near(wgts['0.emb.weight'][idx_house_new], house_wgt)
test_near(wgts['1.decoder.bias'][idx_house_new], house_bias)

We can load the pretrained weights in our model before beginning training

In [19]:
model.load_state_dict(wgts)

<All keys matched successfully>

If we want to apply discriminative learning rates, we need to split our model in different layer groups. Let's have a look at our model

In [20]:
model

SequentialRNN(
  (0): AWD_LSTM(
    (emb): Embedding(60003, 300, padding_idx=1)
    (emb_dp): EmbeddingDropout(
      (emb): Embedding(60003, 300, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (output_dp): RNNDropout()
    (decoder): Linear(in_features=300, out_features=60003, bias=True)
  )
)

In [21]:
model[0].rnns

ModuleList(
  (0): WeightDropout(
    (module): LSTM(300, 300, batch_first=True)
  )
  (1): WeightDropout(
    (module): LSTM(300, 300, batch_first=True)
  )
)

Then we split by doing two groups for each rnn/corresponding dropout, then one last group that contains the embeddings/decoder. This is the one that needs to be trained the most as we may have new embedding vectors.

In [22]:
def lm_splitter(m):
    groups = []
    for i in range(len(m[0].rnns)): groups.append(nn.Sequential(m[0].rnns[i], m[0].hidden_dps[i]))
    groups += [nn.Sequential(m[0].emb, m[0].emb_dp, m[0].input_dp, m[1])]
    return [list(o.parameters()) for o in groups]

First we train with RNNs freezed.

In [23]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(False)

In [24]:
cbfs = [partial(AvgStatsCallback, accuracy_flat),
        CudaCallback, Recorder,
        partial(GradientClipping, clip=0.1),
        partial(RNNTrainer, α=2.,β=1.),
        ProgressCallback]

In [25]:
??RNNTrainer

Init signature: RNNTrainer(α, β)
Docstring:      <no docstring>
Source:        
class RNNTrainer(Callback):
    def __init__(self, α, β): self.α,self.β = α,β

    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]

    def after_loss(self):
        #AR and TAR
        if self.α != 0.:  self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.β * (h[:,1:] - h[:,:-1]).float().pow(2).mean()

    def begin_epoch(self):
        #Shuffle the texts at the beginning of the epoch
        if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()
File:           ~/tutorials/fastai/course-v3/nbs/dl2/exp/nb_12a.py
Type:           type
Subclasses:     


In [26]:
# cbs = [partial(AvgStatsCallback,accuracy_flat),
#        CudaCallback, Recorder,
#        partial(GradientClipping, clip=0.1),
#        partial(RNNTrainer, α=2., β=1.),
#        ProgressCallback]

In [27]:
learn = Learner(model, data, cross_entropy_flat, opt_func=adam_opt(),
               cb_funcs=cbfs, splitter=lm_splitter)

In [28]:
lr = 2e-2
cbsched = sched_1cycle([lr], pct_start=0.5, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [29]:
learn.fit(1, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.469694,0.249359,4.285738,0.263264,11:41


Then the whole model with discriminative Learning rates.

In [29]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(True)

In [30]:
lr = 2e-3
cbsched = sched_1cycle([lr/2., lr/2., lr], pct_start=0.5, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [32]:
learn.fit(10, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,4.251014,0.264398,4.230121,0.269082,12:00
1,4.198551,0.269937,4.191798,0.273234,12:02
2,4.151754,0.274679,4.157451,0.277031,12:02
3,4.110642,0.278739,4.128846,0.279933,11:59
4,4.074635,0.282204,4.104198,0.282238,11:59
5,4.042483,0.285189,4.085981,0.284121,11:59
6,4.014966,0.287529,4.069858,0.285851,11:59
7,3.991425,0.289725,4.058854,0.287003,11:59
8,3.974131,0.291204,4.053309,0.287705,12:00
9,3.965766,0.292051,4.051745,0.287879,11:59


We now have IMDB vocab language model.  
  
We only need to save the encoder(first part of the model) for the classification, as well as the vocabulary used(we will need to use the same in the classification task).

In [ ]:
torch.save(learn.model[0].state_dict(), path/'finetuned_enc.pth')

In [34]:
pickle.dump(vocab, open(path/'vocab_lm.pkl', 'wb'))

In [35]:
torch.save(learn.model.state_dict(), path/'finetuned.pth')

## Classifier

We have to process the data again otherwise pickle will complain. We also have to use the same vocab as the language model.

In [31]:
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))
proc_tok, proc_num, proc_cat = TokenizeProcessor(), NumericalizeProcessor(vocab=vocab), CategoryProcessor()

In [32]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, proc_x=[proc_tok, proc_num], proc_y=proc_cat)

In [34]:
pickle.dump(ll, open(path/'ll_clas.pkl', 'wb'))

In [35]:
ll = pickle.load(open(path/'ll_clas.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [34]:
bs, bptt = 64, 72
data = clas_databunchify(ll, bs)

### Ignore padding

We will use the two utiltiy functions from Pytorch to ignore the padding in the inputs.

In [36]:
#export
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

Let's see how this works: first we grab a batch of the training set.

In [37]:
x, y = next(iter(data.train_dl))

In [38]:
x.size()

torch.Size([64, 3311])

We need to pass to the utility functions the length of our sentences because its applied after the embedding, so we can't see the padding anymore.

In [39]:
x

tensor([[    2,     7,  1178,  ..., 15897,    24,     3],
        [    2,     7,    65,  ...,     1,     1,     1],
        [    2,     7,  4797,  ...,     1,     1,     1],
        ...,
        [    2,   106,    19,  ...,     1,     1,     1],
        [    2,     7,  1237,  ...,     1,     1,     1],
        [    2,     7,  1057,  ...,     1,     1,     1]])

In [40]:
(x == 1).sum(1).size()

torch.Size([64])

In [41]:
lengths = x.size(1) - (x == 1).sum(1)
lengths.size(), lengths[:5]

(torch.Size([64]), tensor([3311, 1951, 1702, 1521, 1429]))

In [42]:
tst_emb = nn.Embedding(len(vocab), 300)

In [43]:
tst_emb(x).shape

torch.Size([64, 3311, 300])

In [44]:
128*70

8960

We create a `PackedSequence` object that contains all our unpadded sequences.

In [45]:
packed = pack_padded_sequence(tst_emb(x), lengths, batch_first=True)

In [46]:
packed.data

tensor([[ 0.2072, -0.3146,  1.5190,  ...,  0.2513, -0.3004, -0.1442],
        [ 0.2072, -0.3146,  1.5190,  ...,  0.2513, -0.3004, -0.1442],
        [ 0.2072, -0.3146,  1.5190,  ...,  0.2513, -0.3004, -0.1442],
        ...,
        [-0.1046,  1.4629,  0.2956,  ..., -0.4439, -0.8925,  1.2392],
        [-1.0641,  0.6144, -1.5101,  ..., -1.2483,  1.6221, -0.2244],
        [ 0.3025, -0.8524,  0.2246,  ...,  0.2291,  0.9697,  0.5775]],
       grad_fn=<PackPaddedSequenceBackward>)

In [47]:
packed.data.shape

torch.Size([78804, 300])

In [48]:
len(packed.batch_sizes)

3311

In [49]:
8960//70

128

This object can be passed to any RNN directly while retaining the speed of CUDNN

In [50]:
tst = nn.LSTM(300, 300, 2)
y, h = tst(packed)

Then we can unpad it with the following function for the other modules.

In [51]:
unpack = pad_packed_sequence(y, batch_first=True)

In [52]:
unpack[0].shape

torch.Size([64, 3311, 300])

In [53]:
unpack[0]

tensor([[[ 0.0246, -0.0059,  0.0010,  ..., -0.0262,  0.0284,  0.0331],
         [ 0.0113, -0.0067, -0.0177,  ..., -0.0376,  0.0392,  0.0321],
         [ 0.0145, -0.0019,  0.0234,  ..., -0.0372,  0.0442,  0.0410],
         ...,
         [-0.0162,  0.0438, -0.0313,  ...,  0.0496,  0.0621,  0.0177],
         [-0.0382,  0.0361, -0.0183,  ...,  0.0461,  0.0694, -0.0130],
         [-0.0122,  0.0221, -0.0132,  ...,  0.0457,  0.0605,  0.0116]],

        [[ 0.0246, -0.0059,  0.0010,  ..., -0.0262,  0.0284,  0.0331],
         [ 0.0113, -0.0067, -0.0177,  ..., -0.0376,  0.0392,  0.0321],
         [-0.0018, -0.0198,  0.0008,  ..., -0.0244,  0.0358,  0.0464],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0246, -0.0059,  0.0010,  ..., -0.0262,  0.0284,  0.0331],
         [ 0.0113, -0.0067, -0.0177,  ..., -0

In [54]:
unpack[1]

tensor([3311, 1951, 1702, 1521, 1429, 1394, 1390, 1371, 1365, 1353, 1351, 1344,
        1344, 1323, 1305, 1296, 1295, 1273, 1270, 1266, 1263, 1263, 1259, 1256,
        1252, 1224, 1219, 1218, 1211, 1203, 1174, 1167, 1158, 1153, 1143, 1138,
        1111, 1110, 1102, 1098, 1098, 1088, 1086, 1083, 1079, 1073, 1073, 1073,
        1062, 1061, 1050, 1048, 1048, 1044, 1042, 1042, 1040, 1033, 1029, 1028,
        1020, 1012, 1009, 1007])

We need to change our model a little bit to use this.

In [55]:
#export
class AWD_LSTM1(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182"
    initrange = 0.1
    
    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                hidden_p = 0.2, input_p = 0.6, embed_p = 0.1, weight_p = 0.5):
        super().__init__()
        self.bs, self.emb_sz, self.n_hid, self.n_layers, self.pad_token = 1, emb_sz, n_hid, n_layers, pad_token
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx = pad_token) # get embedded matrix
        self.emb_dp = EmbeddingDropout(self.emb, embed_p) # apply dropout on it with p=embed_p
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l!= n_layers-1 else emb_sz), 1,
                            batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns]) # apply weight dropout
        self.emb.weight.data.uniform_(-self.initrange, self.initrange) # re-initialize emb weights
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)]) # apply RNN dropout(with seq_dim)

    def forward(self, input):
        bs, sl = input.size()
#         if bs!=self.bs:
#             self.bs = bs
#             self.reset()
        mask = (input == self.pad_token)
        lengths = sl - mask.long().sum(1)
        n_empty = (lengths == 0).sum()
        if n_empty>0:
            input = input[:-n_empty]
            lengths = lengths[:-n_empty]
            self.hidden = [(h[0][:, :input.size(0)], h[1][:, :input.size(0)]) for h in self.hidden]
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden, raw_outputs, outputs = [], [], []
        for l, (rnn, hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output = pack_padded_sequence(raw_output, lengths, batch_first=True)
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            raw_output = pad_packed_sequence(raw_output, batch_first=True)[0]
            raw_outputs.append(raw_output)
            if l!= self.n_layers - 1: raw_output = hid_dp(raw_output)
            new_hidden.append(new_h)
            outputs.append(raw_output)
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs, mask
    
    def _one_hidden(self, l):
        "Return one hidden state"
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()
    
    def reset(self):
        "Reset the hidden states"
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

### Concat Pooling

We will use three things for the classfication head of the model: the last hidden state, the average of all the hidden states and the maximum of all the hidden states. The trick is just to, once again, ignore the padding in the last element/average/maximum.

In [56]:
class Pooling(nn.Module):
    def forward(self, input):
        raw_outputs, outputs, mask = input
        output = outputs[-1]
        lengths = output.size(1) - mask.long().sum(dim=1)
        avg_pool = output.masked_fill(mask[:,:, None], 0).sum(dim=1)
        avg_pool.div_(lengths.type(avg_pool.dtype)[:, None])
        max_pool = output.masked_fill(mask[:,:, None], -float('inf')).max(dim=1)[0]
        x = torch.cat([output[torch.arange(0, output.size(0)), lengths-1], max_pool, avg_pool], 1) # Concat Pooling
        return output, x

In [57]:
emb_sz, nh, nl = 300, 300, 2
tok_pad = vocab.index(PAD)

In [58]:
enc = AWD_LSTM1(len(vocab), emb_sz, n_hid=nh, n_layers=nl, pad_token=tok_pad)
pool = Pooling()
enc.bs = bs
enc.reset()

In [59]:
x, y = next(iter(data.train_dl))
output, c = pool(enc(x))

We can check we have padding with 1s at the end of each text(except the first wich is the longest).

In [60]:
x

tensor([[    2,     7,  1178,  ..., 15897,    24,     3],
        [    2,     7,    65,  ...,     1,     1,     1],
        [    2,     7,  1826,  ...,     1,     1,     1],
        ...,
        [    2,    18,  1830,  ...,     1,     1,     1],
        [    2,     7,    63,  ...,     1,     1,     1],
        [    2,    55,    95,  ...,     1,     1,     1]])

Pytorch puts 0s everywhere we had padding in the `output` when unpacking.

In [61]:
test_near((output.sum(dim=2) == 0).float(), (x==tok_pad).float())

In [62]:
(output.sum(dim=2) == 0).float().size()

torch.Size([64, 3311])

So the last hidden state isn't the last element of 'output'. Let's check if we goteverything right

In [63]:
for i in range(bs):
    length = x.size(1) - (x[i]==1).long().sum()
    out_unpad = output[i, :length]
    test_near(out_unpad[-1], c[i, :300])
    test_near(out_unpad.max(0)[0], c[i, 300:600])
    test_near(out_unpad.mean(0), c[i, 600:])

Our pooling layer properly ignored the padding so now let's group it with a classfier.

In [64]:
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p!=0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

In [65]:
class PoolingLinearClassifier(nn.Module):
    "Create a linear classifier with pooling"
    
    def __init__(self, layers, drops):
        super().__init__()
        mod_layers = []
        activs = [nn.ReLU(inplace=True)] * (len(layers)-2) + [None]
        for n_in, n_out, p, actn in zip(layers[:-1], layers[1:], drops, activs):
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
        self.layers = nn.Sequential(*mod_layers)
        
        
    def forward(self, input):
        raw_outputs, outputs, mask = input
        output = outputs[-1]
        lengths = output.size(1) - mask.long().sum(dim=1)
        avg_pool = output.masked_fill(mask[:,:, None], 0).sum(dim=1)
        avg_pool.div_(lengths.type(avg_pool.dtype)[:, None])
        max_pool = output.masked_fill(mask[:,:, None], -float('inf')).max(dim=1)[0]
        x = torch.cat([output[torch.arange(0, output.size(0)), lengths-1], max_pool, avg_pool], 1) # Concat Pooling
        x = self.layers(x)
        return x

Then we just have to feed our texts to those two blocks,(but we can't give them all at once to the AWD_LSTM(unlike in transformers) or we might get OOM error: we'll go for chunks of bptt length to regularly detach the history of our hidden states.)

In [66]:
def pad_tensor(t, bs, val=0.):
    if t.size(0)<bs:
        return torch.cat([t, val + t.new_zeros(bs-t.size(0), *t.shape[1:])])
    return t

In [67]:
class SentenceEncoder(nn.Module):
    def __init__(self, module, bptt, pad_idx=1):
        super().__init__()
        self.bptt, self.module, self.pad_idx = bptt, module, pad_idx
        
    def concat(self, arrs, bs):
        return [torch.cat([pad_tensor(l[si], bs) for l in arrs], dim=1) for si in range(len(arrs[0]))]
    
    def forward(self, input):
        bs, sl = input.size()
        self.module.bs = bs
        self.module.reset()
        raw_outputs, outputs, masks = [], [], []
        for i in range(0, sl, self.bptt):
            r, o, m = self.module(input[:, i: min(i+self.bptt, sl)])
            masks.append(pad_tensor(m, bs, 1))
            raw_outputs.append(r)
            outputs.append(o)
        return self.concat(raw_outputs, bs), self.concat(outputs, bs), torch.cat(masks, dim=1)

In [68]:
def get_text_classifier(vocab_sz, emb_sz, n_hid, n_layers, n_out, pad_token, bptt, output_p=0.4, hidden_p=0.2,
                      input_p= 0.6, embed_p=0.1, weight_p=0.5, layers=None, drops=None):
    "To create a full AWD-LSTM"
    rnn_enc = AWD_LSTM1(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                      hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = SentenceEncoder(rnn_enc, bptt)
    if layers is None: layers = [50]
    if drops is None: drops = [0.1] * len(layers)
    layers = [3 * emb_sz] + layers + [n_out]
    drops = [output_p] + drops
    return SequentialRNN(enc, PoolingLinearClassifier(layers, drops))

In [69]:
emb_sz, n, nl = 300, 300, 2
dps = tensor([0.4, 0.3, 0.4, 0.05]) * 0.25
model = get_text_classifier(len(vocab), emb_sz, nh, nl, 2, 1, bptt, *dps)

## Training

We load our pretrained encoder and freeze it.

In [70]:
def class_splitter(m):
    enc = m[0].module
    groups = [nn.Sequential(enc.emb, enc.emb_dp, enc.input_dp)]
    for i in range(len(enc.rnns)): groups.append(nn.Sequential(enc.rnns[i], enc.hidden_dps[i]))
    groups.append(m[1])
    return [list(o.parameters()) for o in groups]

In [71]:
for p in model[0].parameters(): p.requires_grad_(False)

In [72]:
cbs = [partial(AvgStatsCallback, accuracy),
      CudaCallback, Recorder,
      partial(GradientClipping, clip=0.1),
      ProgressCallback]

In [73]:
model[0].module.load_state_dict(torch.load(path/'finetuned_enc.pth'))

<All keys matched successfully>

In [74]:
learn = Learner(model, data, F.cross_entropy, opt_func=adam_opt(), cb_funcs=cbs, splitter=class_splitter)

In [75]:
lr = 1e-2
cbsched = sched_1cycle([lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [76]:
learn.fit(1, cbs = cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.344405,0.851680,0.272190,0.886120,00:47


In [78]:
for p in model[0].module.rnns[-1].parameters(): p.requires_grad_(True)

In [79]:
lr = 5e-3
cbsched = sched_1cycle([lr/2., lr/2., lr/2., lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [80]:
learn.fit(1, cbs = cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.270406,0.886720,0.220873,0.910800,00:59


In [81]:
for p in model[0].parameters(): p.requires_grad_(True)

In [82]:
lr = 1e-3
cbsched = sched_1cycle([lr/8., lr/4., lr/2., lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [83]:
learn.fit(4, cbs = cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.215812,0.912800,0.198032,0.921520,01:13
1,0.191641,0.923000,0.191263,0.925960,01:12
2,0.158608,0.937200,0.186516,0.926800,01:12
3,0.137127,0.947040,0.186727,0.927880,01:12


In [84]:
x, y = next(iter(data.valid_dl))

Predicting on the padded batch or on the individual unpadded samples gives the same results

In [86]:
pred_batch = learn.model.eval()(x.cuda())

In [85]:
pred_ind = []
for inp in x:
    length = x.size(1) - (inp == 1).long().sum()
    inp = inp[:length]
    pred_ind.append(learn.model.eval()(inp[None].cuda()))

In [87]:
assert near(pred_batch, torch.cat(pred_ind))